In [1]:
import os
import sys

sys.path.insert(0, '../src')

from decimal import Decimal
from datetime import datetime

from the_pitch.data import Query
from the_pitch.simulators import PitchSimulator, Scorer
from the_pitch.strategies import StrategyFactory

In [2]:
symbols=['DAL', 'AAL', 'ALK', 'BA']

if not os.path.exists('./data/airlines.csv'):
    df = Query.get_pricing_data(source='yahoo', symbols=symbols, start='2017-01-01', end='2021-01-01')
    df.to_csv('./data/airlines.csv')

dataset = Query.load_dataset(csv = './data/airlines.csv', now='2018-06-01')

In [3]:
simulator = PitchSimulator(dataset)

In [4]:
## get strategy, indicators
iterations = [
    StrategyFactory.create_sma_cross_price(
        id='TS1',
        symbols=symbols,
        period1=20,
    ),
    StrategyFactory.create_price_revert_to_sma(
        id='TS2',
        symbols=symbols,
        period1=20,
        pl=Decimal('.12'),
        pm=Decimal('.05')
    ),
    StrategyFactory.create_sma_cross_sma(
        id='TS3',
        symbols=symbols,
        period1=50,
        period2=200
    ),
]

runs = []
for strategy, indicators in iterations:

    ## run simulator
    operations = simulator.run(
        indicators=indicators,
        strategies=[ strategy ]
    )

    runs.append(operations)

In [5]:
display(
    runs[0]['TS1']['AAL'][0:4]
)
display(
    runs[1]['TS2']['AAL'][0:4]
)
display(
    runs[2]['TS3']['AAL'][0:4]
)

[Position("TS1", "AAL", AssetType(1), 1, 43.22, 2018-06-11 00:00:00, Side(1)),
 Position("TS1", "AAL", AssetType(1), 1, 42.49, 2018-06-12 00:00:00, Side(2)),
 Position("TS1", "AAL", AssetType(1), 1, 37.23, 2018-07-23 00:00:00, Side(1)),
 Position("TS1", "AAL", AssetType(1), 1, 36.66, 2018-07-24 00:00:00, Side(2))]

[Position("TS2", "AAL", AssetType(1), 1, 32.93, 2018-10-09 00:00:00, Side(1)),
 Position("TS2", "AAL", AssetType(1), 1, 34.43, 2018-10-31 00:00:00, Side(2)),
 Position("TS2", "AAL", AssetType(1), 1, 30.29, 2018-12-21 00:00:00, Side(1)),
 Position("TS2", "AAL", AssetType(1), 1, 33.43, 2019-01-18 00:00:00, Side(2))]

[Position("TS3", "AAL", AssetType(1), 1, 17.63, 2020-12-08 00:00:00, Side(1))]

In [6]:
for run in runs:
    scores = Scorer.score(run)
    print(scores)

{'TS1': {'DAL': Decimal('18.69'), 'AAL': Decimal('-4.88'), 'ALK': Decimal('-9.32'), 'BA': Decimal('-50.01')}}
{'TS2': {'AAL': Decimal('5.03'), 'DAL': Decimal('-8.53'), 'ALK': Decimal('-8.65'), 'BA': Decimal('-140.89')}}
{'TS3': {'DAL': Decimal('-11.02'), 'ALK': Decimal('-21.46'), 'BA': Decimal('-52.38'), 'AAL': Decimal('0.0')}}


In [7]:
## expected,
{'TS1': {'DAL': Decimal('18.69'), 'AAL': Decimal('-4.88'), 'ALK': Decimal('-9.32'), 'BA': Decimal('-50.01')}}
{'TS2': {'AAL': Decimal('5.03'), 'DAL': Decimal('-8.53'), 'ALK': Decimal('-8.65'), 'BA': Decimal('-140.89')}}
{'TS3': {'DAL': Decimal('-11.02'), 'ALK': Decimal('-21.46'), 'BA': Decimal('-52.38'), 'AAL': Decimal('0.0')}}

{'TS3': {'DAL': Decimal('-11.02'),
  'ALK': Decimal('-21.46'),
  'BA': Decimal('-52.38'),
  'AAL': Decimal('0.0')}}